In [2]:
import sys; sys.path.insert(0, '../../invert')
from invert.forward import get_info, create_forward_model
import mne
import pickle as pkl
import os

os.makedirs("forward_models", exist_ok=True)

# Settings

In [ ]:
sampling_coarse = "ico4"
sampling_fine = "oct6"

kind="biosemi128"
n_chans = 128

info = get_info(kind=kind)  # contains the channel locations

fn = f"forward_models/{n_chans}_ch_info.pkl"
with open(fn, 'wb') as f:
    pkl.dump(info, f)

In [ ]:
subject = 'fsaverage'
fs_dir = mne.datasets.fetch_fsaverage(verbose=0)
subjects_dir = os.path.dirname(fs_dir)
trans = os.path.join(fs_dir, 'bem', 'fsaverage-trans.fif')
src = os.path.join(fs_dir, 'bem', 'fsaverage-ico-5-src.fif')
bem = os.path.join(fs_dir, 'bem', 'fsaverage-5120-5120-5120-bem-sol.fif')
verbose=0

# Clean coarse Forward Model

In [ ]:
ratio = 80
conductivity = [0.33, 0.33/ratio, 0.33]

ico = 4
surfaces = mne.make_bem_model(subject, ico=ico, conductivity=conductivity)
bem = mne.make_bem_solution(surfaces)

src = mne.setup_source_space(subject, spacing=sampling_coarse, surface='white',
                                        subjects_dir=subjects_dir, add_dist=False,
                                        n_jobs=-1, verbose=verbose)
fwd = mne.make_forward_solution(info, trans=trans, src=src,
                                    bem=bem, eeg=True, meg=True, mindist=5.0, n_jobs=-1,
                                    verbose=verbose)
fwd = mne.convert_forward_solution(fwd, surf_ori=True, force_fixed=True,
                                            use_cps=True, verbose=verbose)
fname = f"forward_models/{n_chans}_ch_coarse_{ratio}_ratio-fwd.fif"
mne.write_forward_solution(fname, fwd)

# Clean fine Forward Model [0.33 0.0042 0.33] (1:80)

In [ ]:
ratio = 80
conductivity = [0.33, 0.33/ratio, 0.33]

ico = 4
surfaces = mne.make_bem_model(subject, ico=ico, conductivity=conductivity)
bem = mne.make_bem_solution(surfaces)

src = mne.setup_source_space(subject, spacing=sampling_fine, surface='white',
                                        subjects_dir=subjects_dir, add_dist=False,
                                        n_jobs=-1, verbose=verbose)
fwd = mne.make_forward_solution(info, trans=trans, src=src,
                                    bem=bem, eeg=True, meg=True, mindist=5.0, n_jobs=-1,
                                    verbose=verbose)
fwd = mne.convert_forward_solution(fwd, surf_ori=True, force_fixed=True,
                                            use_cps=True, verbose=verbose)
fname = f"forward_models/{n_chans}_ch_fine_{ratio}_ratio-fwd.fif"
mne.write_forward_solution(fname, fwd)

# Clean fine Forward Model [0.33, 0.0066, 0.33] (1:50)


In [ ]:
ratio = 50
conductivity = [0.33, 0.33/ratio, 0.33]

ico = 4
surfaces = mne.make_bem_model(subject, ico=ico, conductivity=conductivity)
bem = mne.make_bem_solution(surfaces)

src = mne.setup_source_space(subject, spacing=sampling_fine, surface='white',
                                        subjects_dir=subjects_dir, add_dist=False,
                                        n_jobs=-1, verbose=verbose)
fwd = mne.make_forward_solution(info, trans=trans, src=src,
                                    bem=bem, eeg=True, meg=True, mindist=5.0, n_jobs=-1,
                                    verbose=verbose)
fwd = mne.convert_forward_solution(fwd, surf_ori=True, force_fixed=True,
                                            use_cps=True, verbose=verbose)
fname = f"forward_models/{n_chans}_ch_fine_{ratio}_ratio-fwd.fif"
mne.write_forward_solution(fname, fwd)

# Clean fine Forward Model [0.332, 0.0165, 0.332] (1:20)


In [ ]:
ratio = 20
conductivity = [0.33, 0.33/ratio, 0.33]
ico = 4
surfaces = mne.make_bem_model(subject, ico=ico, conductivity=conductivity)
bem = mne.make_bem_solution(surfaces)

src = mne.setup_source_space(subject, spacing=sampling_fine, surface='white',
                                        subjects_dir=subjects_dir, add_dist=False,
                                        n_jobs=-1, verbose=verbose)
fwd = mne.make_forward_solution(info, trans=trans, src=src,
                                    bem=bem, eeg=True, meg=True, mindist=5.0, n_jobs=-1,
                                    verbose=verbose)
fwd = mne.convert_forward_solution(fwd, surf_ori=True, force_fixed=True,
                                            use_cps=True, verbose=verbose)
fname = f"forward_models/{n_chans}_ch_fine_{ratio}_ratio-fwd.fif"
mne.write_forward_solution(fname, fwd)

# MEG-type Errors

## Translation Posterior

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
evokeds = []
evoked = mne.EvokedArray(fwd["sol"]["data"][:, 0][:, np.newaxis], info).set_eeg_reference()
evokeds.append(evoked)
direction = "posterior"
translation_list = [1e-3, 2e-3]  # 1-2 mm in meters
for translation in translation_list:
    info_trans = info.copy()
    for i in range(len(info_trans["chs"])):
        info_trans["chs"][i]["loc"][1] -= translation 

    fwd = create_forward_model(sampling=sampling_fine, info=info_trans)
    evoked = mne.EvokedArray(fwd["sol"]["data"][:, 0][:, np.newaxis], info).set_eeg_reference()
    evokeds.append(evoked)
    fwd.comment = dict(sampling=sampling_fine, n_channels=n_chans, error_type="translation posterior", error_magnitude=int(translation*1e3))
    fn = f"forward_models/{n_chans}_ch_info_translation-{int(translation*1e3)}mm-{direction}.pkl"
    with open(fn, 'wb') as f:
        pkl.dump(info_trans, f)
    
    mne.write_forward_solution(f"forward_models/{n_chans}_ch_{sampling_fine}-translation-{int(translation*1e3)}mm-{direction}-fwd.fif", fwd, overwrite=True, verbose=0)

In [ ]:
# info_trans = info.copy()
montage1 = info.get_montage()
montage1.plot()

montage2 = info_trans.get_montage()
montage2.plot()

## Translation Dorsal

In [ ]:
direction = "dorsal"
translation_list = [1e-3, 2e-3]  # 1-2 mm in meters
for translation in translation_list:
    info_trans = info.copy()

    for i in range(len(info_trans["chs"])):
        info_trans["chs"][i]["loc"][2] += translation 

    fwd = create_forward_model(sampling=sampling_fine, info=info_trans)
    fwd.comment = dict(sampling=sampling_fine, n_channels=n_chans, error_type="translation dorsal", error_magnitude=int(translation*1e3))
    fn = f"forward_models/{n_chans}_ch_info_translation-{int(translation*1e3)}mm-{direction}.pkl"
    with open(fn, 'wb') as f:
        pkl.dump(info_trans, f)
    mne.write_forward_solution(f"forward_models/{n_chans}_ch_{sampling_fine}-translation-{int(translation*1e3)}mm-{direction}-fwd.fif", fwd, overwrite=True, verbose=0)

In [ ]:
# info_trans = info.copy()
montage1 = info.get_montage()
montage1.plot()

montage2 = info_trans.get_montage()
montage2.plot()

## Translation Right

In [ ]:
translation_list = [1e-3, 2e-3]  # 1-2 mm in meters
direction = "right"
for translation in translation_list:
    info_trans = info.copy()
    for i in range(len(info_trans["chs"])):
        info_trans["chs"][i]["loc"][0] += translation

    fwd = create_forward_model(sampling=sampling_fine, info=info_trans)
    fwd.comment = dict(sampling=sampling_fine, n_channels=n_chans, error_type="translation right", error_magnitude=int(translation*1e3))
    fn = f"forward_models/{n_chans}_ch_info_translation-{int(translation*1e3)}mm-{direction}.pkl"
    with open(fn, 'wb') as f:
        pkl.dump(info_trans, f)
    mne.write_forward_solution(f"forward_models/{n_chans}_ch_{sampling_fine}-translation-{int(translation*1e3)}mm-{direction}-fwd.fif", fwd, overwrite=True, verbose=0)

In [ ]:
# info_trans = info.copy()
montage1 = info.get_montage()
montage1.plot()

montage2 = info_trans.get_montage()
montage2.plot()

## Rotation right

In [ ]:
import numpy as np
import math

def rotate_coordinates(coords, axis, degree):
    theta = math.radians(degree)  # Convert degrees to radians
    
    # Depending on the axis, define the rotation matrix
    if axis == 'x':
        rotation_matrix = np.array([
            [1, 0, 0],
            [0, np.cos(theta), -np.sin(theta)],
            [0, np.sin(theta), np.cos(theta)]
        ])
    elif axis == 'y':
        rotation_matrix = np.array([
            [np.cos(theta), 0, np.sin(theta)],
            [0, 1, 0],
            [-np.sin(theta), 0, np.cos(theta)]
        ])
    elif axis == 'z':
        rotation_matrix = np.array([
            [np.cos(theta), -np.sin(theta), 0],
            [np.sin(theta), np.cos(theta), 0],
            [0, 0, 1]
        ])
    else:
        raise ValueError("Invalid axis. Choose from 'x', 'y', or 'z'.")
    
    # Rotate each coordinate
    rotated_coords = [np.dot(rotation_matrix, np.array(coord).T).T for coord in coords]
    return rotated_coords

for i in range(len(info_trans["chs"])):
        info_trans["chs"][i]["loc"][2] += translation
degrees_list = [1, 2]
direction = "right"
for degrees in degrees_list:
    # pos = np.stack([p["r"] for p in info['dig']], axis=0)
    pos = np.stack([p["loc"][:3] for p in info["chs"]], axis=0)
    pos_rotated = rotate_coordinates(pos, 'y', degrees)
    info_rotate = info.copy()
    for i, new_coord in enumerate(pos_rotated):
        info_rotate["chs"][i]["loc"][:3] = new_coord 

    fwd = create_forward_model(sampling=sampling_fine, info=info_rotate)
    fwd.comment = dict(sampling=sampling_fine, n_channels=n_chans, error_type="rotation right", error_magnitude=int(degrees))
    fn = f"forward_models/{n_chans}_ch_info_rotation-{degrees}deg-{direction}.pkl"
    with open(fn, 'wb') as f:
        pkl.dump(info_rotate, f)
    mne.write_forward_solution(f"forward_models/{n_chans}_ch_{sampling_fine}-rotation-{degrees}deg-{direction}-fwd.fif", fwd, overwrite=True, verbose=0)

In [ ]:
# info_trans = info.copy()
montage1 = info.get_montage()
montage1.plot()

montage2 = info_rotate.get_montage()
montage2.plot()

## Rotation up

In [ ]:
import numpy as np
import math

def rotate_coordinates(coords, axis, degree):
    theta = math.radians(degree)  # Convert degrees to radians
    
    # Depending on the axis, define the rotation matrix
    if axis == 'x':
        rotation_matrix = np.array([
            [1, 0, 0],
            [0, np.cos(theta), -np.sin(theta)],
            [0, np.sin(theta), np.cos(theta)]
        ])
    elif axis == 'y':
        rotation_matrix = np.array([
            [np.cos(theta), 0, np.sin(theta)],
            [0, 1, 0],
            [-np.sin(theta), 0, np.cos(theta)]
        ])
    elif axis == 'z':
        rotation_matrix = np.array([
            [np.cos(theta), -np.sin(theta), 0],
            [np.sin(theta), np.cos(theta), 0],
            [0, 0, 1]
        ])
    else:
        raise ValueError("Invalid axis. Choose from 'x', 'y', or 'z'.")
    
    # Rotate each coordinate
    rotated_coords = [np.dot(rotation_matrix, np.array(coord).T).T for coord in coords]
    return rotated_coords

degrees_list = [1, 2]
direction = "up"
for degrees in degrees_list:
    # pos = np.stack([p["r"] for p in info['dig']], axis=0)
    pos = np.stack([p["loc"][:3] for p in info["chs"]], axis=0)
    pos_rotated = rotate_coordinates(pos, 'x', degrees)
    info_rotate = info.copy()
    for i, new_coord in enumerate(pos_rotated):
        info_rotate["chs"][i]["loc"][:3] = new_coord 

    fwd = create_forward_model(sampling=sampling_fine, info=info_rotate)
    fwd.comment = dict(sampling=sampling_fine, n_channels=n_chans, error_type="rotation up", error_magnitude=int(degrees))
    fn = f"forward_models/{n_chans}_ch_info_rotation-{degrees}deg-{direction}.pkl"
    with open(fn, 'wb') as f:
        pkl.dump(info_rotate, f)
    mne.write_forward_solution(f"forward_models/{n_chans}_ch_{sampling_fine}-rotation-{degrees}deg-{direction}-fwd.fif", fwd, overwrite=True, verbose=0)

In [ ]:
# info_trans = info.copy()
montage1 = info.get_montage()
montage1.plot()

montage2 = info_rotate.get_montage()
montage2.plot()

# 256 Channels

In [ ]:
kind="biosemi256"
n_chans = 256
info256 = get_info(kind=kind)

fwd = create_forward_model(sampling=sampling_fine, info=info256)
fwd.comment = dict(sampling=sampling_fine, n_channels=n_chans, error_type=None, error_magnitude=0)
fn = f"forward_models/{n_chans}_ch_info.pkl"
with open(fn, 'wb') as f:
    pkl.dump(info256, f)
mne.write_forward_solution(f"forward_models/{n_chans}_ch_{sampling_fine}-fwd.fif", fwd, overwrite=True, verbose=0)
fwd